# 21. 연산자 오버로드(2)

- 개선점: ndarray 인스터스 및 수치 데이터와 함께 사용하기
- Variable 인스턴스, ndarry 인스턴스, int, float 등도 함께 사용할 수 있게 수정해 봅시다. 

## 21.1 ndarray와 함께 사용하기

- a * np.array(2.0) 코드를 만나면 ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환하기
- as_variable 편의 함수를 만들어서 인수로 주어진 객체를 Variable 인스턴스로 변환해 주는 함수 구현하기

In [1]:
def as_variable(obj):
    if isinstance(obj, Variable):    # 인수 obj가 Variable 인스턴스면 그대로 반환
        return obj
    return Variable(obj)   # 그렇지 않으면 Variable 인스턴스로 변환하여 반환

In [2]:
class Function: # DeZero에서 사용하는 모든 함수(연산)은 Function 클래스를 상속하므로 실제 연산은 Function __call__메서드에서 이뤄짐
    def __call__(self, *inputs):   # 따라서 __call__메서드에 가한 수정은 DeZero에서 사용하는 모든 함수에 적용됨
        inputs = [as_variable(x) for x in inputs]   # 인수 inputs에 담긴 각각의 원소 x를 Variable 인스턴스로 변환

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)

In [5]:
x = Variable(np.array(2.0))    # ndarray 인스턴스가 Variable 인스턴스로 자동 변환 됨 
y = x + np.array(3.0)          # 이제 ndarray와 Variable 함께 사용할 수 있게 됨
print(y)

variable(5.0)


## 21.2 float, int와 함께 사용하기

- 파이썬의 float와 int, 그리고 np.float64과 np.int64 같은 타입과도 함께 사용할 수 있도록 하겠습니다. 
- x가 Variable 인스턴스일 때, x + 3.0 같은 코드를 실행할 수 있도록 하려면 어떻게 해야 할까요?

In [6]:
def add(x0, x1):
    x1 = as_array(x1)      # x1이 float나 int일 경우, ndarray 인스턴스로 변환 
    return Add()(x0, x1)   # 이후 ndarray 인스턴스는 Function 클래스에서 Variable 인스턴스로 변환

In [7]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


- 이와 같이 float와 Variable 인스턴스를 조합한 계산이 가능해짐
- add 함수 외에도 mul 같은 다른 함수도 같은 방식으로 수정할 수 있음
- 수정하고 나면 +나 *로 Variable 인스턴스, float, int를 조합하여 계산할 수 있음
- 지금의 방식에는 두 가지 문제가 남아 있음

## 21.3 문제점 1: 첫번째 인수가 float나 int인 경우

- 현재 DeZero는 x * 2.0을 제대로 실행할 수 있지만 2.0 * x 실행하면 오류가 납니다. 
    - 연산자 왼쪽에 있는 2.0의 \_\_mul\_\_ 메서드를 호출하려 시도한다
    - 하지만 2.0은 float 타입이므로 \_\_mul\_\_ 메서드는 구현되어 있지 않다. 
    - 다음은 * 연산자 오른쪽에 있는 x의 특수 메서드를 호출하려 시도한다
    - x가 오른쪽에 있기 때문에 (\_\_mul\_\_ 메서드 대신) \_\_rmul\_\_ 메서드를 호출하려 시도한다.
    - 하지만 Variable 인스턴스에는 \_\_rmul\_\_ 메서드가 구현되어 있지 않다. 
- 핵심은 * 같은 이항 연산자의 경우 피연산자(항)의 위치에 따라 호출되는 특수 메서드가 다르다는 것입니다. 
    - 곱셉의 경우 피연산자가 좌항이면 \_\_mul\_\_ 메서드가 호출되고, 우항이면 \_\_rmul\_\_ 메서드가 호출됩니다. 
- 따라서 이번 문제는 \_\_rmul\_\_ 메서드를 구현하면  해결됩니다. 
    - 이 때 \_\_rmul\_\_ 메서드의 인수는 아래처럼 전달됩니다. 
    
![title](./image/그림21-1.png)

- 위 그림과 같이 \_\_rmul\_\_(self, other)의 인수 중 self는 자신인 x에 대응하고, other는 다른 쪽 항이 2.0에 대응합니다. 
- 곱셈에서는 좌항과 우항을 바꿔도 결과가 같기 때문에 둘을 구별할 필요가 없습니다. 
- 덧셉도 마찬가지이므로 +와 $*$의 특수 메서드는 아래처럼 설정하면 됩니다. 

In [9]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [10]:
x = Variable(np.array(2.0)) # 이제 Variable 인스턴스와 float, int를 함께 사용할 수 있습니다. 
y = 3.0 * x + 1.0
print(y)

variable(7.0)


## 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우

- 남은 문제는 ndarray 인스턴스가 좌항이고 Variable 인스턴스가 우항인 경우입니다. 

In [11]:
x = Variable(np.array(1.0)) # 좌항 ndarray 인스턴스의 __add__ 메서드가 호출되는데, 
y = np.array([2.0]) + x     # 우항인 Variable 인스턴스의 __radd__ 메서스가 호출되길 원함. 

In [12]:
# 그러려면 '연산 우선순위'를 지정해야 함. Variable 인스턴스 속성에 __array_priority__를 추가하고 그 값을 큰 정수로 설정함
class Variable:               # Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스 연산자 우선순위보다 높일 수 있다. 
    __array_priority__ = 200  # 그 결과 좌항이 ndarray 인스턴스라 해도 우항인 Variable 인스턴스의 연산자 메서드가 우선적으로 호출됨

In [13]:
# 21장 전체 코드

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

variable(5.0)
variable(5.0)
variable(7.0)
